In [1]:
'''
__author__ = 'Alex Cheng'
惩罚逻辑回归
cannot run out
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets,linear_model
from sklearn.metrics import roc_curve,auc,roc_auc_score
from math import sqrt

In [2]:
target_url = ("http://archive.ics.uci.edu/ml/machine-learning-databases/"
             "undocumented/connectionist-bench/sonar/sonar.all-data")
df = pd.read_csv(target_url,header=None,sep=",",prefix='V')
df.head()

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [3]:
xList  = np.array(df.iloc[:,:-1]) # 属性数组
labels = []
names = df.columns
print(df.iloc[:,-1].value_counts()) # M:111, R:97

 # 二分类值转换为实数
for label in df.iloc[:,-1]:
    if(label == 'M'):
        labels.append(1.0)
    else:
        labels.append(0.0)


M    111
R     97
Name: V60, dtype: int64


In [4]:
# 正则化columns in x and labels
nrows = len(xList)
ncols = len(xList[0])

# 计算means and variance
xMeans = []
xSD = []
for i in range(ncols):
    col = [xList[j][i] for j in range(nrows)]
    mean = sum(col)/nrows
    xMeans.append(mean)
    colDiff = sum((col - mean) **2)
    
    colDiff2 = [(xList[j][i] - mean) for j in range(nrows) ]
    sumSq = sum([colDiff2[i]  * colDiff2[i] for i in range(nrows)])
    stdDev2 = sqrt(sumSq/nrows)
    
    stdDev = sqrt(colDiff/nrows)
    xSD.append(stdDev)
#print(xMeans,'\n',xSD)
#print(stdDev == stdDev2) # broadcast广播变量的方式，发现结果一样


# 正则化xList
xNorm = []
for i in range(nrows):
    #rowNorm = ([xList[i] - xMeans]) / xSD  #列表不支持
    rowNorm = [(xList[i][j] - xMeans[j]) / xSD[j] for j in range(ncols)]
    xNorm.append(rowNorm)

# 正则化 labels
meanLable = sum(labels)/ nrows
sdLabel = sqrt(sum([ (labels[i] - meanLable) * (labels[i] - meanLable) for i in range(nrows)]) / nrows)
labelNorm = [ (labels[i] - meanLable) / sdLabel for i in range(nrows) ]

In [27]:
from math import sqrt, fabs,exp
def S(z,gamma):
    if gamma >= fabs(z):
        return 0.0
    if z>0.0:
        return z-gamma
    else:
        return z+gamma

def Pr(b0,b,x):
    n = len(x)
    sum = b0
    for i in range(n):
        sum += b[i]*x[i]
        if sum < -100: sum =-100
    return 1.0/(1.0+exp(-sum))

In [34]:
# 不要 normalize lable仅仅计算均值
# 惩罚 逻辑回归
# 初始化 probability and weight
alpha = 0.8
sumWxr = [0.0] * ncols
sumWxx = [0.0] * ncols
sumWr = 0.0
sumW = 0.0
#Y = np.array(labelNorm)
#X = np.array(xNorm)

# 计算betas的起点
for row in range(nrows):
    p = meanLable # 标签均值
    w = p * (1.0 - p)
    # 逻辑回归的残差
    r = (labels[row] - p) # 每一行的
    x = xNorm[row]
    sumWxr = [sumWxr[i] + w * x[i] * r    for i in range(ncols)]
    sumWxx = [sumWxx[i] + w * x[i] * x[i] for i in range(ncols)]
    sumWr = sumWr + w * r
    sumW  = sumW + w

#print('sumWxr: \n', sumWxr)
avgWxr = [sumWxr[i]/nrows for i in range(ncols)]
avgWxx = [sumWxx[i]/nrows for i in range(ncols)]

maxWxr = 0.0
for i in range(ncols):
    val = abs(avgWxr[i])
    if val > maxWxr:
        maxWxr = val # 找绝对值最大的

lam = maxWxr/alpha
# 初始化 对应beta的lambda值
beta = [0.0] * ncols
beta0 = sumWr / sumW
betaMat = []
betaMat.append(list(beta))

beta0List = []
beta0List.append(beta0)

# begin iteration
steps = 100
lamMult = 0.93
nzList = []
for step in range(steps):
    lam = lam * lamMult
    # 使用增量改变在betas中去控制内循环
    
    betaIRLS = list(beta)
    beta0IRLS = beta0
    distIRLS = 100.0
    iterIRLS = 0
    while distIRLS > 0.01:
        iterIRLS += 1
        iterInner = 0.0
        
        betaInner = list(betaIRLS)
        beta0Inner = beta0IRLS
        distInner = 100.0
        while distInner >0.01:
            iterInner += 1
            if iterInner > 100: break
            
            # 更新
            betaStart = list(betaInner)
            for col in range(ncols):
                sumWxr = 0.0
                sumWxx = 0.0
                sumWr = 0.0
                sumW = 0.0
                
                for row in range(nrows):
                    x = list(xNorm[row])
                    y = labels[row]
                    p = Pr(beta0IRLS, betaIRLS, x)
                    if abs(p) < 1e-5:
                        p=0.0
                        w=1e-5
                    elif abs(1.0-p) < 1e-5:
                        p=1.0
                        w = 1e-5
                    else:
                        w = p *(1.0-p)
                    
                    z = (y-p)/w + beta0IRLS + sum([x[i] * betaIRLS[i] for i in range(ncols)])
                    r = z - beta0Inner - sum([x[i] * betaInner[i] for i in range(ncols)])
                    sumWxr += w* x[col] * r
                    sumWxx += w * x[col] * x[col]
                    sumWr += w * r
                    sumW += w
                
                avgWxr = sumWxr / nrows
                avgWxx = sumWxx / nrows
                
                beta0Inner = beta0Inner + sumWr/sumW
                uncBeta = avgWxr + avgWxx * betaInner[col]
                betaInner[col] = S(uncBeta, lam * alpha) / (avgWxx + lam * (1.0 - alpha))
            
            sumDiff = sum([abs(betaInner[n] - betaStart[n]) for n in range(ncols)])
            sumBeta = sum([abs(betaInner[n]) for n in range(ncols)])
            distInner = sumDiff/sumBeta
        
        # check change in betaMiddle to see if IRLS is converged
        a = sum([abs(betaIRLS[i]) for i in range(ncols)])
        b = sum([abs(betaIRLS[i]) for i in range(ncols)])
        distIRLS = a/(b+0.0001)
        dBeta = [betaInner[i] - betaIRLS[i] for i in range(ncols)]
        gradStep = 1.0
        temp = [betaIRLS[i] + gradStep * dBeta[i] for i in range(ncols)]
        betaIRLS = list(temp)
    
    beta = list(betaIRLS)
    beta0 = beta0IRLS
    betaMat.append(list(beta))
    beta0List.append(beta0)
    
    nzBeta = [index for index in range(ncols) if beta[index] != 0.0]
    for q in nzBeta:
        if not(q in nzList):
            nzList.append(q)
    
nameList = [names[nzList[i]] for i in range(len(nzList))]
    
print('Attributes ordered by how early they enter the model')
print(nameList)
for i in range(ncols):
    coefCurve = [betaMat[k][i] for k in range(steps)]
    xaxis = range(steps)
    plt.plot(xaxis,coefCurve)
plt.xlabel('Steps Taken')
plt.ylabel('Coefficient values')
plt.show()
    
            



KeyboardInterrupt: 

In [19]:
#labelNorm

In [20]:
#labels